# Number Detector

The goal of this little Project is to get familiar with PyTorch to create machine learning models. For this simple introduction I am going to use the following Dataset of hand painted Numbers between 0 and 9, to Train Test and Validate the MLM.

Dataset: https://github.com/SebLague/Mnist-data-numpy-format

In [9]:
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader

dataPath = "../data/Mnist-HanddrawnNumbers-Dataset/mnist.npz"
data = np.load(dataPath)

trainImg = data["training_images"]
trainLbl = data["training_labels"]
testImg = data["test_images"]
testLbl = data["test_labels"]
valiImg = data["validation_images"]
valiLbl = data["validation_labels"]

t_trainImg = torch.tensor(trainImg)
t_trainLbl = torch.tensor(trainLbl)   
t_testImg = torch.tensor(testImg)
t_testLbl = torch.tensor(testLbl)
t_valiImg = torch.tensor(valiImg)
t_valiLbl = torch.tensor(valiLbl)

trainDS = DataLoader(TensorDataset(t_trainImg, t_trainLbl))
testDS = DataLoader(TensorDataset(t_testImg, t_testLbl))
valiDS = DataLoader(TensorDataset(t_valiImg, t_valiLbl))

## Creating the Neural Network

The next step after Loading the Data is to create the Layers of the Neural Network

In [12]:
import torch
import torch.nn as nn

class MyNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(29, 9, 5)
        self.pool = nn.MaxPool2d(1)


    def forward(self, elem):
        elem = torch.flatten(elem, 1)
        return elem
    
net = MyNet()



## Training

In [14]:
from tqdm import tqdm
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), 0.01, 0.5)
celoss = nn.CrossEntropyLoss()

EPOCHS = 2
for e in range(EPOCHS):
    loss = 0.0
    for i, d in enumerate(tqdm(trainDS, f"EPOCH: ({e + 1}/{EPOCHS})")):
        img, lbl = d
        optimizer.zero_grad()
        out = net.forward(img)
        loss = celoss(out, lbl)
        loss.backward()
        optimizer.step()


EPOCH: (1/2):   0%|          | 0/50000 [00:01<?, ?it/s]


RuntimeError: 0D or 1D target tensor expected, multi-target not supported